# Diffusion Module - Quickstart Guide

**Session 2: Closed-Form Diffusion Solutions**

This notebook demonstrates:
- Constant-source diffusion (erfc solution)
- Limited-source diffusion (Gaussian solution)
- Junction depth calculation
- Sheet resistance estimation
- Two-step diffusion process
- Temperature and time dependencies

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# Add module to path
sys.path.insert(0, str(Path.cwd().parent))

from core.erfc import (
    constant_source_profile,
    limited_source_profile,
    junction_depth,
    sheet_resistance_estimate,
    two_step_diffusion,
    quick_profile_constant_source,
    quick_profile_limited_source,
)

# Plot styling
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✓ Imports successful")

## 1. Constant-Source Diffusion

Constant-source diffusion occurs when the surface concentration is held constant during the process. This is typical for:
- Dopant gas flow processes (BBr₃, POCl₃)
- Solid-source diffusion with infinite supply

The solution uses the complementary error function:

$$N(x,t) = C_s \cdot \text{erfc}\left(\frac{x}{2\sqrt{Dt}}\right) + N_{A0}$$

In [ ]:
# Boron constant-source diffusion at 1000°C
x = np.linspace(0, 1000, 1000)  # Depth (nm)
t = 30 * 60  # 30 minutes -> seconds
T = 1000  # °C

# Boron parameters
D0_B = 0.76  # cm²/s
Ea_B = 3.46  # eV
Cs = 1e20  # Surface concentration (atoms/cm³)
NA0 = 1e15  # Background concentration (atoms/cm³)

# Calculate profile
C = constant_source_profile(x, t, T, D0_B, Ea_B, Cs, NA0)

# Calculate junction depth
xj = junction_depth(C, x, NA0)

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Linear scale
ax1.plot(x, C, 'b-', linewidth=2)
ax1.axhline(NA0, color='r', linestyle='--', label=f'Background: {NA0:.1e} cm⁻³')
ax1.axvline(xj, color='g', linestyle='--', label=f'Junction: {xj:.1f} nm')
ax1.set_xlabel('Depth (nm)')
ax1.set_ylabel('Concentration (cm⁻³)')
ax1.set_title('Boron Constant-Source Diffusion\n1000°C, 30 min')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Log scale
ax2.semilogy(x, C, 'b-', linewidth=2)
ax2.axhline(NA0, color='r', linestyle='--', label=f'Background: {NA0:.1e} cm⁻³')
ax2.axvline(xj, color='g', linestyle='--', label=f'Junction: {xj:.1f} nm')
ax2.set_xlabel('Depth (nm)')
ax2.set_ylabel('Concentration (cm⁻³)')
ax2.set_title('Boron Constant-Source Diffusion (Log Scale)\n1000°C, 30 min')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(1e14, 1e21)

plt.tight_layout()
plt.show()

print(f"Surface concentration: {C[0]:.2e} cm⁻³")
print(f"Junction depth: {xj:.1f} nm")
print(f"Concentration at 100nm: {C[100]:.2e} cm⁻³")

### Time Dependence

Let's examine how the diffusion profile evolves with time:

In [ ]:
# Multiple time points
times = [10, 20, 30, 45, 60]  # minutes
colors = plt.cm.viridis(np.linspace(0, 1, len(times)))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

junction_depths = []

for i, t_min in enumerate(times):
    t = t_min * 60  # Convert to seconds
    C = constant_source_profile(x, t, T, D0_B, Ea_B, Cs, NA0)
    xj = junction_depth(C, x, NA0)
    junction_depths.append(xj)
    
    ax1.plot(x, C, color=colors[i], linewidth=2, label=f'{t_min} min (xⱼ={xj:.0f} nm)')
    ax2.semilogy(x, C, color=colors[i], linewidth=2, label=f'{t_min} min')

ax1.axhline(NA0, color='r', linestyle='--', alpha=0.5, label='Background')
ax1.set_xlabel('Depth (nm)')
ax1.set_ylabel('Concentration (cm⁻³)')
ax1.set_title('Time Evolution of Boron Diffusion\n1000°C, Constant Source')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.axhline(NA0, color='r', linestyle='--', alpha=0.5)
ax2.set_xlabel('Depth (nm)')
ax2.set_ylabel('Concentration (cm⁻³)')
ax2.set_title('Time Evolution (Log Scale)')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(1e14, 1e21)

plt.tight_layout()
plt.show()

# Plot junction depth vs time
plt.figure(figsize=(8, 5))
times_array = np.array(times)
junction_array = np.array(junction_depths)
plt.plot(times_array, junction_array, 'o-', linewidth=2, markersize=8)

# Fit to sqrt(t) dependence
from scipy.optimize import curve_fit
def sqrt_fit(t, a):
    return a * np.sqrt(t)
popt, _ = curve_fit(sqrt_fit, times_array, junction_array)
t_fit = np.linspace(0, 60, 100)
plt.plot(t_fit, sqrt_fit(t_fit, *popt), 'r--', alpha=0.7, 
         label=f'√t fit: xⱼ = {popt[0]:.2f}√t')

plt.xlabel('Time (minutes)')
plt.ylabel('Junction Depth (nm)')
plt.title('Junction Depth vs Time\nBoron @ 1000°C')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Junction depth scales as √t with coefficient {popt[0]:.2f} nm/√min")

### Temperature Dependence

Higher temperatures lead to deeper diffusion profiles:

In [ ]:
# Multiple temperatures
temperatures = [900, 950, 1000, 1050, 1100]  # °C
t = 30 * 60  # 30 minutes
colors = plt.cm.plasma(np.linspace(0, 1, len(temperatures)))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

junction_depths_T = []

for i, T_curr in enumerate(temperatures):
    C = constant_source_profile(x, t, T_curr, D0_B, Ea_B, Cs, NA0)
    xj = junction_depth(C, x, NA0)
    junction_depths_T.append(xj)
    
    ax1.plot(x, C, color=colors[i], linewidth=2, 
             label=f'{T_curr}°C (xⱼ={xj:.0f} nm)')
    ax2.semilogy(x, C, color=colors[i], linewidth=2, label=f'{T_curr}°C')

ax1.axhline(NA0, color='k', linestyle='--', alpha=0.5, label='Background')
ax1.set_xlabel('Depth (nm)')
ax1.set_ylabel('Concentration (cm⁻³)')
ax1.set_title('Temperature Dependence of Boron Diffusion\n30 minutes, Constant Source')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.axhline(NA0, color='k', linestyle='--', alpha=0.5)
ax2.set_xlabel('Depth (nm)')
ax2.set_ylabel('Concentration (cm⁻³)')
ax2.set_title('Temperature Dependence (Log Scale)')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(1e14, 1e21)

plt.tight_layout()
plt.show()

## 2. Limited-Source Diffusion

Limited-source diffusion occurs when a fixed dose is deposited and redistributed. This is typical for:
- Ion implantation followed by annealing
- Drive-in after pre-deposition

The solution is a Gaussian:

$$N(x,t) = \frac{Q}{\sqrt{\pi D t}} \exp\left(-\frac{x^2}{4Dt}\right) + N_{A0}$$

In [ ]:
# Phosphorus limited-source diffusion at 950°C
x_lim = np.linspace(0, 800, 800)
t_lim = 20 * 60  # 20 minutes
T_lim = 950  # °C

# Phosphorus parameters
D0_P = 3.85  # cm²/s
Ea_P = 3.66  # eV
Q = 1e14  # Total dose (atoms/cm²)
NA0 = 1e15  # Background

# Calculate profile
C_lim = limited_source_profile(x_lim, t_lim, T_lim, D0_P, Ea_P, Q, NA0)

# Calculate junction depth
xj_lim = junction_depth(C_lim, x_lim, NA0)

# Calculate sheet resistance
Rs = sheet_resistance_estimate(C_lim, x_lim, dopant_type='n')

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(x_lim, C_lim, 'g-', linewidth=2)
ax1.axhline(NA0, color='r', linestyle='--', label=f'Background: {NA0:.1e} cm⁻³')
ax1.axvline(xj_lim, color='b', linestyle='--', label=f'Junction: {xj_lim:.1f} nm')
ax1.set_xlabel('Depth (nm)')
ax1.set_ylabel('Concentration (cm⁻³)')
ax1.set_title(f'Phosphorus Limited-Source Diffusion\n950°C, 20 min, Q={Q:.1e} cm⁻²\nRₛ = {Rs:.1f} Ω/□')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.semilogy(x_lim, C_lim, 'g-', linewidth=2)
ax2.axhline(NA0, color='r', linestyle='--', label='Background')
ax2.axvline(xj_lim, color='b', linestyle='--', label=f'Junction: {xj_lim:.1f} nm')
ax2.set_xlabel('Depth (nm)')
ax2.set_ylabel('Concentration (cm⁻³)')
ax2.set_title('Phosphorus Limited-Source (Log Scale)')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(1e14, 1e21)

plt.tight_layout()
plt.show()

print(f"Peak concentration: {C_lim[0]:.2e} cm⁻³")
print(f"Junction depth: {xj_lim:.1f} nm")
print(f"Sheet resistance: {Rs:.1f} Ω/□")

### Time Evolution of Limited Source

As time increases, the dose spreads and peak concentration decreases:

In [ ]:
# Multiple time points for limited source
times_lim = [5, 10, 20, 40, 60]  # minutes
colors = plt.cm.cool(np.linspace(0, 1, len(times_lim)))

fig, ax = plt.subplots(figsize=(10, 6))

for i, t_min in enumerate(times_lim):
    t = t_min * 60
    C = limited_source_profile(x_lim, t, T_lim, D0_P, Ea_P, Q, NA0)
    ax.semilogy(x_lim, C, color=colors[i], linewidth=2, 
                label=f'{t_min} min (peak={C[0]:.2e} cm⁻³)')

ax.axhline(NA0, color='k', linestyle='--', alpha=0.5, label='Background')
ax.set_xlabel('Depth (nm)')
ax.set_ylabel('Concentration (cm⁻³)')
ax.set_title(f'Time Evolution of Phosphorus Limited-Source Diffusion\n950°C, Q={Q:.1e} cm⁻²')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(1e14, 1e21)

plt.tight_layout()
plt.show()

print("Notice: As time increases, peak decreases but profile spreads deeper")

## 3. Two-Step Diffusion Process

A common process sequence:
1. **Pre-deposition**: Short time at moderate temperature with dopant source
2. **Drive-in**: Longer time (often higher temperature) without dopant source

This creates a controlled, deep junction with desired profile shape.

In [ ]:
# Two-step boron diffusion
# Step 1: Pre-dep at 900°C for 15 min
# Step 2: Drive-in at 1100°C for 30 min

x_two = np.linspace(0, 1500, 1500)

C_predep, C_drivein = two_step_diffusion(
    x_two,
    t1=15*60, T1=900,  # Pre-dep
    t2=30*60, T2=1100,  # Drive-in
    D0=D0_B, Ea=Ea_B,
    Cs=1e20, NA0=1e15
)

# Calculate junction depths
xj_predep = junction_depth(C_predep, x_two, NA0)
xj_drivein = junction_depth(C_drivein, x_two, NA0)

# Calculate sheet resistances
Rs_predep = sheet_resistance_estimate(C_predep, x_two, 'p')
Rs_drivein = sheet_resistance_estimate(C_drivein, x_two, 'p')

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(x_two, C_predep, 'b-', linewidth=2, label=f'After Pre-dep (xⱼ={xj_predep:.0f} nm)')
ax1.plot(x_two, C_drivein, 'r-', linewidth=2, label=f'After Drive-in (xⱼ={xj_drivein:.0f} nm)')
ax1.axhline(NA0, color='k', linestyle='--', alpha=0.5, label='Background')
ax1.set_xlabel('Depth (nm)')
ax1.set_ylabel('Concentration (cm⁻³)')
ax1.set_title('Two-Step Boron Diffusion\nPre-dep: 900°C/15min, Drive-in: 1100°C/30min')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.semilogy(x_two, C_predep, 'b-', linewidth=2, label='After Pre-dep')
ax2.semilogy(x_two, C_drivein, 'r-', linewidth=2, label='After Drive-in')
ax2.axhline(NA0, color='k', linestyle='--', alpha=0.5, label='Background')
ax2.set_xlabel('Depth (nm)')
ax2.set_ylabel('Concentration (cm⁻³)')
ax2.set_title('Two-Step Boron Diffusion (Log Scale)')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(1e14, 1e21)

plt.tight_layout()
plt.show()

print("\nPre-deposition:")
print(f"  Junction depth: {xj_predep:.1f} nm")
print(f"  Sheet resistance: {Rs_predep:.1f} Ω/□")
print(f"  Peak concentration: {C_predep[0]:.2e} cm⁻³")

print("\nAfter drive-in:")
print(f"  Junction depth: {xj_drivein:.1f} nm")
print(f"  Sheet resistance: {Rs_drivein:.1f} Ω/□")
print(f"  Peak concentration: {C_drivein[0]:.2e} cm⁻³")

print(f"\nJunction deepened by: {xj_drivein - xj_predep:.1f} nm")

## 4. Comparison: Different Dopants

Different dopants have different diffusion rates:

In [ ]:
# Compare boron, phosphorus, and arsenic
dopants = {
    'Boron': (0.76, 3.46, 'blue'),
    'Phosphorus': (3.85, 3.66, 'green'),
    'Arsenic': (0.066, 3.44, 'red'),
}

x_comp = np.linspace(0, 800, 800)
t_comp = 30 * 60
T_comp = 1000
Cs = 1e20
NA0 = 1e15

fig, ax = plt.subplots(figsize=(10, 6))

for dopant_name, (D0, Ea, color) in dopants.items():
    C = constant_source_profile(x_comp, t_comp, T_comp, D0, Ea, Cs, NA0)
    xj = junction_depth(C, x_comp, NA0)
    
    ax.semilogy(x_comp, C, color=color, linewidth=2, 
                label=f'{dopant_name} (xⱼ={xj:.0f} nm)')

ax.axhline(NA0, color='k', linestyle='--', alpha=0.5, label='Background')
ax.set_xlabel('Depth (nm)')
ax.set_ylabel('Concentration (cm⁻³)')
ax.set_title('Comparison of Dopant Diffusion Rates\n1000°C, 30 min, Constant Source')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(1e14, 1e21)

plt.tight_layout()
plt.show()

print("Diffusion rate order (fastest to slowest):")
print("Phosphorus > Boron > Arsenic")
print("\nThis is important for process design!")

## 5. Sheet Resistance Analysis

Sheet resistance is a key electrical parameter for device design:

In [ ]:
# Sheet resistance vs dose for phosphorus
doses = np.logspace(12, 15, 20)  # 1e12 to 1e15 atoms/cm²
Rs_values = []

x_rs = np.linspace(0, 500, 500)
t_rs = 20 * 60
T_rs = 950

for Q in doses:
    C = limited_source_profile(x_rs, t_rs, T_rs, D0_P, Ea_P, Q, NA0)
    Rs = sheet_resistance_estimate(C, x_rs, 'n')
    Rs_values.append(Rs)

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(doses, Rs_values, 'o-', linewidth=2, markersize=8)
ax.set_xlabel('Dose (atoms/cm²)')
ax.set_ylabel('Sheet Resistance (Ω/□)')
ax.set_title('Sheet Resistance vs Implant Dose\nPhosphorus, 950°C, 20 min anneal')
ax.grid(True, alpha=0.3, which='both')

# Add typical ranges
ax.axhspan(10, 100, alpha=0.2, color='green', label='Source/Drain (10-100 Ω/□)')
ax.axhspan(100, 1000, alpha=0.2, color='yellow', label='Gate/Interconnect (100-1000 Ω/□)')
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

print("Typical sheet resistance targets:")
print("  Source/Drain: 10-100 Ω/□")
print("  Gate/Poly: 20-50 Ω/□")
print("  Resistor: 100-10,000 Ω/□")

## 6. Summary and Key Insights

### Constant-Source Diffusion
- Profile shape: erfc (complementary error function)
- Junction depth scales as √(D·t)
- Maintains high surface concentration
- Used for: Pre-deposition, doped oxide sources

### Limited-Source Diffusion
- Profile shape: Gaussian
- Total dose conserved
- Peak concentration decreases with time
- Used for: Ion implant annealing, drive-in

### Temperature Effects
- D(T) follows Arrhenius: D = D₀·exp(-Eₐ/(k·T))
- Higher T → deeper diffusion
- Small ΔT causes large ΔD (exponential sensitivity)

### Process Design Considerations
1. Choose appropriate source type
2. Select time/temperature to achieve target xⱼ
3. Verify sheet resistance meets specs
4. Consider two-step process for better control
5. Account for dopant-specific diffusion rates

## Next Steps

**Session 3** will add numerical solvers for:
- Concentration-dependent diffusivity
- Arbitrary boundary conditions
- Complex time-temperature profiles
- Validation against analytical solutions

**Try experimenting with:**
- Different dopant combinations
- Various time/temperature profiles
- Calculate your target junction depths
- Optimize for desired sheet resistance